# This notebook will be used for the capstone project

In [1]:
### Set up project

In [2]:
import pandas as pd
import numpy as np
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


print('Hello Capstone Project Course!')

Hello Capstone Project Course!


### Fetch Toronto neighbourhood table from Wiki

In [3]:
## URL of table
wiki_url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
### scrape wiki page
df_original = pd.read_html(wiki_url, header=0)[0]
print(df_original.head(5))
df = df_original

  Postal Code           Borough              Neighbourhood
0         M1A      Not assigned               Not assigned
1         M2A      Not assigned               Not assigned
2         M3A        North York                  Parkwoods
3         M4A        North York           Victoria Village
4         M5A  Downtown Toronto  Regent Park, Harbourfront


## Pre-processing
### Remove cells where Borough is not assigned

In [4]:
df = df[df['Borough'] != 'Not assigned']
print(df.head())

  Postal Code           Borough                                Neighbourhood
2         M3A        North York                                    Parkwoods
3         M4A        North York                             Victoria Village
4         M5A  Downtown Toronto                    Regent Park, Harbourfront
5         M6A        North York             Lawrence Manor, Lawrence Heights
6         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government


### Merge Neighbourhoods with same postal code

In [5]:
print('Unique postal codes: {}'.format(len(df['Postal Code'].unique())))

Unique postal codes: 103


In [6]:
PostalCodeGroups = df.groupby('Postal Code').count()
PostalCodeMultiples = PostalCodeGroups[PostalCodeGroups['Borough'] > 1].index.values
print(PostalCodeGroups.head())

for i in PostalCodeMultiples:
    df_pcm = df[df['Postal Code'] == i]
    
    borough = df_pcm.iloc[0]['Borough']
    joined = ','.join(df_pcm['Neighborhood'].tolist())
    
    df = df[df['Postal Code'] != i]
    df = df.append({'Postal Code': i,
                   'Borough': borough,
                   'Neighbourhood': joined},
                  ignore_index = True)
    
print(df.head())
print(df.shape)

             Borough  Neighbourhood
Postal Code                        
M1B                1              1
M1C                1              1
M1E                1              1
M1G                1              1
M1H                1              1
  Postal Code           Borough                                Neighbourhood
2         M3A        North York                                    Parkwoods
3         M4A        North York                             Victoria Village
4         M5A  Downtown Toronto                    Regent Park, Harbourfront
5         M6A        North York             Lawrence Manor, Lawrence Heights
6         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government
(103, 3)


Set Neighbourhood as Borough if unassigned

In [7]:
### shape of dataframe
### if neighborhood - not assigned
### --> neighbourhood = borough
idx = df[df['Neighbourhood'] == 'Not assigned'].index.values

for i in idx:
    df.loc[i, 'Neighbourhood'] = df.loc[i, 'Borough']
    


In [8]:
df.shape

(103, 3)

In [9]:
data = pd.read_csv("Geospatial_Coordinates.csv") 
# Preview the first 5 lines of the loaded data 
data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Load geospatial coordinates

In [10]:
df_coord = pd.read_csv("Geospatial_Coordinates.csv") 
df_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merge Lat,Lon Data

In [11]:
df = pd.merge(df, df_coord, on='Postal Code')
print(df)

    Postal Code           Borough  \
0           M3A        North York   
1           M4A        North York   
2           M5A  Downtown Toronto   
3           M6A        North York   
4           M7A  Downtown Toronto   
..          ...               ...   
98          M8X         Etobicoke   
99          M4Y  Downtown Toronto   
100         M7Y      East Toronto   
101         M8Y         Etobicoke   
102         M8Z         Etobicoke   

                                         Neighbourhood   Latitude  Longitude  
0                                            Parkwoods  43.753259 -79.329656  
1                                     Victoria Village  43.725882 -79.315572  
2                            Regent Park, Harbourfront  43.654260 -79.360636  
3                     Lawrence Manor, Lawrence Heights  43.718518 -79.464763  
4          Queen's Park, Ontario Provincial Government  43.662301 -79.389494  
..                                                 ...        ...        ...  
98

### End of part 2

In [12]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

In [13]:
address = 'TORONTO'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The lati & long coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The lati & long coordinate of Toronto are 43.6534817, -79.3839347.


### Visualize all Toronto neighbourhoods

In [14]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [15]:
dtt = df[df['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
dtt.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


### Look at downtown Toronto

In [16]:
address = 'Downtown TORONTO'

geolocator = Nominatim(user_agent="dtt_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The lati & long coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The lati & long coordinate of Downtown Toronto are 43.6541737, -79.38081164513409.


### clustering of Downtown Toronto

In [17]:
# create map of T using latitude and longitude values
map_dt = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(dtt['Latitude'], dtt['Longitude'], dtt['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dt)  
    
map_dt

### Load Foursquare info

In [18]:
CLIENT_ID = 'DO2DZ0V5ZCOSSAOPBBBOP1GDPMUFVVYG55UV45DQOFELBZTH' # your Foursquare ID
CLIENT_SECRET = '1PIX4SXCFIEKXPV5JPGYN3GCDLANHLFLO2L0MYRDBRGW44RD' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DO2DZ0V5ZCOSSAOPBBBOP1GDPMUFVVYG55UV45DQOFELBZTH
CLIENT_SECRET:1PIX4SXCFIEKXPV5JPGYN3GCDLANHLFLO2L0MYRDBRGW44RD


### Explore first neighbourhood

In [19]:
dtt.loc[0, 'Neighbourhood']

'Regent Park, Harbourfront'

In [20]:
neighborhood_latitude = dtt.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = dtt.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = dtt.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


### Check top 100 venues

In [21]:
#<!-- The correct answer is:
LIMIT = 100 # limit of number of venues returned by Foursquare API
#-->

#<!--
radius = 500 # define radius
#-->

#<!--
#\\\\ # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL
#--> 

'https://api.foursquare.com/v2/venues/explore?&client_id=DO2DZ0V5ZCOSSAOPBBBOP1GDPMUFVVYG55UV45DQOFELBZTH&client_secret=1PIX4SXCFIEKXPV5JPGYN3GCDLANHLFLO2L0MYRDBRGW44RD&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=100'

In [22]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '601961382c351f569a584f8c'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 44,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '53b8466a498e83df908c3f21',
       'name': 'Tandem Coffee',
       'location': {'address': '368 King St E',
        'crossStreet': 'at Trinity St',
        'lat': 43.65355870959944,
        'lng': -79.36180945913513,
        'labeledLatLngs': [{'label': 'display',
 

### function that extracts the category of the venue


In [23]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [24]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-24-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Tandem Coffee,Coffee Shop,43.653559,-79.361809
1,Roselle Desserts,Bakery,43.653447,-79.362017
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Impact Kitchen,Restaurant,43.656369,-79.356980


In [25]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

44 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Toronto


In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [27]:
dtt_venues = getNearbyVenues(names=dtt['Neighbourhood'],
                                   latitudes=dtt['Latitude'],
                                   longitudes=dtt['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [28]:
print(dtt_venues.shape)
dtt_venues.head()

(1214, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [29]:
dtt_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,55,55,55,55,55,55
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,60,60,60,60,60,60
Christie,16,16,16,16,16,16
Church and Wellesley,77,77,77,77,77,77
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront East, Union Station, Toronto Islands",100,100,100,100,100,100


In [30]:
print('There are {} uniques categories.'.format(len(dtt_venues['Venue Category'].unique())))

There are 205 uniques categories.


## Neighbourhood Analysis

In [31]:
# one hot encoding
dtt_onehot = pd.get_dummies(dtt_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dtt_onehot['Neighbourhood'] = dtt_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [dtt_onehot.columns[-1]] + list(dtt_onehot.columns[:-1])
dtt_onehot = dtt_onehot[fixed_columns]

dtt_onehot.head()

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
dtt_onehot.shape

(1214, 206)

In [33]:
dtt_grouped = dtt_onehot.groupby('Neighbourhood').mean().reset_index()
dtt_grouped

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.00,0.018182,0.000000,0.00000,0.000000,0.000000,0.000000
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
2,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.00,0.016667,0.000000,0.00000,0.016667,0.000000,0.016667
3,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
4,Church and Wellesley,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012987,0.000000,0.00,...,0.012987,0.012987,0.00,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.025974
5,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.00,...,0.000000,0.000000,0.00,0.00,0.020000,0.000000,0.00000,0.010000,0.000000,0.000000
6,"First Canadian Place, Underground city",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.00,...,0.010000,0.000000,0.00,0.01,0.010000,0.000000,0.00000,0.010000,0.000000,0.000000
7,"Garden District, Ryerson",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.020000,0.000000,0.00,0.00,0.000000,0.010000,0.01000,0.010000,0.000000,0.000000
8,"Harbourfront East, Union Station, Toronto Islands",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.05,...,0.010000,0.000000,0.00,0.01,0.010000,0.000000,0.00000,0.010000,0.000000,0.000000
9,"Kensington Market, Chinatown, Grange Park",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.00,0.00,0.065574,0.000000,0.04918,0.016393,0.000000,0.000000


In [34]:
dtt_grouped.shape

(19, 206)

In [35]:
num_top_venues = 5

for hood in dtt_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = dtt_grouped[dtt_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0     Coffee Shop  0.09
1    Cocktail Bar  0.05
2        Beer Bar  0.04
3  Farmers Market  0.04
4      Restaurant  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0   Airport Service  0.18
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3           Airport  0.06
4       Coffee Shop  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.18
1  Italian Restaurant  0.05
2      Sandwich Place  0.05
3                Café  0.05
4     Bubble Tea Shop  0.03


----Christie----
           venue  freq
0  Grocery Store  0.25
1           Café  0.19
2           Park  0.12
3    Candy Store  0.06
4      Nightclub  0.06


----Church and Wellesley----
                  venue  freq
0           Coffee Shop  0.09
1   Japanese Restaurant  0.06
2      Sushi Restaurant  0.06
3               Gay Bar  0.04
4  Fast Food Restaurant  0.0

In [36]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [37]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = dtt_grouped['Neighbourhood']

for ind in np.arange(dtt_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dtt_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Farmers Market,Seafood Restaurant,Beer Bar,Restaurant,Bakery,Breakfast Spot,Beach
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Airport,Bar,Harbor / Marina,Rental Car Location,Sculpture Garden,Boutique,Boat or Ferry
2,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Bubble Tea Shop,Salad Place,Burger Joint,Indian Restaurant,Diner,Modern European Restaurant
3,Christie,Grocery Store,Café,Park,Restaurant,Baby Store,Italian Restaurant,Athletics & Sports,Candy Store,Nightclub,Coffee Shop
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Fast Food Restaurant,Yoga Studio,Men's Store,Café,Hotel


## 4. Cluster Neighborhoods


In [38]:
# set number of clusters
kclusters = 5
dtt_grouped_clustering = dtt_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dtt_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 1, 2, 1, 1, 1, 1, 1, 4], dtype=int32)

In [39]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

dtt_merged = dtt

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
dtt_merged = dtt_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

dtt_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Bakery,Park,Pub,Café,Theater,Breakfast Spot,Restaurant,Farmers Market,Beer Store
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Sushi Restaurant,Yoga Studio,Fast Food Restaurant,Restaurant,Portuguese Restaurant,Park,Nightclub,Music Venue,Mexican Restaurant
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Clothing Store,Coffee Shop,Cosmetics Shop,Café,Japanese Restaurant,Hotel,Middle Eastern Restaurant,Bubble Tea Shop,Pizza Place,Lingerie Store
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Café,Coffee Shop,Gastropub,Cocktail Bar,American Restaurant,Department Store,Hotel,Seafood Restaurant,Restaurant,Italian Restaurant
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Coffee Shop,Cocktail Bar,Cheese Shop,Farmers Market,Seafood Restaurant,Beer Bar,Restaurant,Bakery,Breakfast Spot,Beach


In [40]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dtt_merged['Latitude'], dtt_merged['Longitude'], dtt_merged['Neighbourhood'], dtt_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters


In [41]:
dtt_merged.loc[dtt_merged['Cluster Labels'] == 0, dtt_merged.columns[[1] + list(range(5, dtt_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Downtown Toronto,0,Airport Service,Airport Lounge,Airport Terminal,Airport,Bar,Harbor / Marina,Rental Car Location,Sculpture Garden,Boutique,Boat or Ferry


In [42]:
dtt_merged.loc[dtt_merged['Cluster Labels'] == 1, dtt_merged.columns[[1] + list(range(5, dtt_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Coffee Shop,Bakery,Park,Pub,Café,Theater,Breakfast Spot,Restaurant,Farmers Market,Beer Store
1,Downtown Toronto,1,Coffee Shop,Sushi Restaurant,Yoga Studio,Fast Food Restaurant,Restaurant,Portuguese Restaurant,Park,Nightclub,Music Venue,Mexican Restaurant
2,Downtown Toronto,1,Clothing Store,Coffee Shop,Cosmetics Shop,Café,Japanese Restaurant,Hotel,Middle Eastern Restaurant,Bubble Tea Shop,Pizza Place,Lingerie Store
3,Downtown Toronto,1,Café,Coffee Shop,Gastropub,Cocktail Bar,American Restaurant,Department Store,Hotel,Seafood Restaurant,Restaurant,Italian Restaurant
4,Downtown Toronto,1,Coffee Shop,Cocktail Bar,Cheese Shop,Farmers Market,Seafood Restaurant,Beer Bar,Restaurant,Bakery,Breakfast Spot,Beach
5,Downtown Toronto,1,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Bubble Tea Shop,Salad Place,Burger Joint,Indian Restaurant,Diner,Modern European Restaurant
7,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Thai Restaurant,Deli / Bodega,Clothing Store,Bakery,Gym,American Restaurant,Sushi Restaurant
8,Downtown Toronto,1,Coffee Shop,Aquarium,Hotel,Café,Scenic Lookout,Restaurant,Brewery,Italian Restaurant,Fried Chicken Joint,Bar
9,Downtown Toronto,1,Coffee Shop,Hotel,Café,Restaurant,Salad Place,Seafood Restaurant,American Restaurant,Japanese Restaurant,Italian Restaurant,Sushi Restaurant
10,Downtown Toronto,1,Coffee Shop,Restaurant,Café,Hotel,American Restaurant,Italian Restaurant,Gym,Cocktail Bar,Seafood Restaurant,Deli / Bodega


In [43]:
dtt_merged.loc[dtt_merged['Cluster Labels'] == 2, dtt_merged.columns[[1] + list(range(5, dtt_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Downtown Toronto,2,Grocery Store,Café,Park,Restaurant,Baby Store,Italian Restaurant,Athletics & Sports,Candy Store,Nightclub,Coffee Shop


In [44]:
dtt_merged.loc[dtt_merged['Cluster Labels'] == 3, dtt_merged.columns[[1] + list(range(5, dtt_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,3,Park,Playground,Trail,Cupcake Shop,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner


In [45]:
dtt_merged.loc[dtt_merged['Cluster Labels'] == 4, dtt_merged.columns[[1] + list(range(5, dtt_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Downtown Toronto,4,Café,Bookstore,Japanese Restaurant,Bar,Bakery,Sushi Restaurant,Sandwich Place,College Gym,Beer Store,Beer Bar
12,Downtown Toronto,4,Café,Vegetarian / Vegan Restaurant,Coffee Shop,Mexican Restaurant,Vietnamese Restaurant,Farmers Market,Gaming Cafe,Caribbean Restaurant,Dessert Shop,Grocery Store
